In [5]:

from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
model = load_model('../model/epoch_058.hdf5')
encoder_model = Model(inputs=model.input, outputs=model.layers[7].output)

encoder_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 96, 160, 1)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 48, 80, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 40, 32)        4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 20, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 10, 128)        73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 5, 256)         295168    
_________________________________________________________________
flatten (Flatten)            (None, 3840)              0   

In [1]:
import numpy as np
all_feats = np.load('../../../../../../data/palm_all_wld_aug_feats.npy')
all_feats.shape

(3000, 95, 125)

In [2]:
import numpy as np
all_feats = np.load('../../../../../../data/palm_all_mc_aug_feats.npy')
all_feats.shape

(3000, 394, 522)

In [4]:
import numpy as np
all_feats = np.load('../../../../../../data/palm_all_imgs_aug_prep.npy')
all_feats.shape

(3000, 394, 522)

In [7]:
import numpy as np
all_feats = np.load('../../../../../../../PUT-wrist/data/palm_all_rlt_aug_feats.npy')
all_feats.shape

(3000, 236, 313)

In [3]:
from tensorflow.keras.layers import Activation, Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping , CSVLogger, ReduceLROnPlateau

from tensorflow.keras import backend as K

import numpy as np
np.random.seed(1994)

# Network parameters
#390, 682
input_shape = (96, 128, 1)#95, 125
input_shape = (224, 320, 1)#236, 313
input_shape = (384, 512, 1)#394,522
batch_size = 128
kernel_size = 3
latent_dim = 1000
# Encoder/Decoder number of CNN layers and filters per layer
layer_filters = [16, 32, 64,128,256]


def my_loss(y_true, y_pred):
    MSE = K.sum(K.square(y_pred - y_true), axis=-1)
    MAE = K.sum(K.abs(y_pred - y_true), axis=-1)
    return MSE + 0.2 * MAE


# Build the Autoencoder Model
def getModel():
    # First build the Encoder Model
    inputs = Input(shape=input_shape, name='encoder_input')
    x = inputs
    # Stack of Conv2D blocks
    # Notes:
    # 1) Use Batch Normalization before ReLU on deep networks
    # 2) Use MaxPooling2D as alternative to strides>1
    # - faster but not as good as strides>1
    for filters in layer_filters:
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=2,
                   activation='relu',
                   padding='same')(x)

    # Shape info needed to build Decoder Model
    shape = K.int_shape(x)

    # Generate the latent vector
    x = Flatten()(x)
    latent = Dense(latent_dim, activation='relu', name='latent_vector')(x)

#     # Instantiate Encoder Model
#     encoder = Model(inputs, latent, name='encoder')
#     encoder.summary()
#
#     # Build the Decoder Model
#     latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
#    x = Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
    
    x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(latent)
    x = Reshape((shape[1], shape[2], shape[3]))(x)

    # Stack of Transposed Conv2D blocks
    # Notes:
    # 1) Use Batch Normalization before ReLU on deep networks
    # 2) Use UpSampling2D as alternative to strides>1
    # - faster but not as good as strides>1
    for filters in layer_filters[::-1]:
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=2,
                            activation='relu',
                            padding='same')(x)

    x = Conv2DTranspose(filters=1,
                        kernel_size=kernel_size,
                        padding='same')(x)

    outputs = Activation('sigmoid', name='decoder_output')(x)
    

    # Instantiate Decoder Model
    model = Model(inputs, outputs, name='decoder')
    
    optim=Adam(lr=1e-3, decay=0.0)
    model.compile(loss = "binary_crossentropy",#[my_loss, ssim_loss], loss_weights = [1, 0.075],#'mse'
                  optimizer = optim,
                  metrics   = [metrics.mse,
                               metrics.mae
                               ])
    model.summary()
    return model

gmodel = getModel()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 384, 512, 1)]     0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 192, 256, 16)      160       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 96, 128, 32)       4640      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 48, 64, 64)        18496     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 24, 32, 128)       73856     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 12, 16, 256)       295168    
_________________________________________________________________
flatten_2 (Flatten)          (None, 49152)             0   